<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.09375
Current benign train loss: 2.321268320083618

Current batch: 100
Current benign train accuracy: 0.09375
Current benign train loss: 5.680656909942627

Current batch: 200
Current benign train accuracy: 0.1640625
Current benign train loss: 2.4031424522399902

Current batch: 300
Current benign train accuracy: 0.1796875
Current benign train loss: 2.27785587310791

Total benign train accuarcy: 16.948
Total benign train loss: 1469.9220396280289

[ Test epoch: 0 ]

Test accuarcy: 18.83
Test average loss: 0.022328117311000824
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.203125
Current benign train loss: 2.274493932723999

Current batch: 100
Current benign train accuracy: 0.234375
Current benign train loss: 2.079233407974243

Current batch: 200
Current benign train accuracy: 0.28125
Current benign train loss: 2.0117135047912598

Current batch: 300
Current benign train accuracy: 0.2


Current batch: 200
Current benign train accuracy: 0.6640625
Current benign train loss: 0.8822281956672668

Current batch: 300
Current benign train accuracy: 0.6640625
Current benign train loss: 0.8284949660301208

Total benign train accuarcy: 62.914
Total benign train loss: 407.66604965925217

[ Test epoch: 13 ]

Test accuarcy: 15.16
Test average loss: 0.11209312210083008
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.5546875
Current benign train loss: 1.4957046508789062

Current batch: 100
Current benign train accuracy: 0.671875
Current benign train loss: 0.9208616018295288

Current batch: 200
Current benign train accuracy: 0.5546875
Current benign train loss: 1.091522216796875

Current batch: 300
Current benign train accuracy: 0.6328125
Current benign train loss: 1.000218152999878

Total benign train accuarcy: 64.89
Total benign train loss: 383.94159281253815

[ Test epoch: 14 ]

Test accuarcy: 30.36
Test average loss: 0.03408409276008606
Model 


Total benign train accuarcy: 73.39
Total benign train loss: 297.9226515889168

[ Test epoch: 26 ]

Test accuarcy: 20.89
Test average loss: 0.08055170335769653
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.6953125
Current benign train loss: 0.740088701248169

Current batch: 100
Current benign train accuracy: 0.703125
Current benign train loss: 0.9510278105735779

Current batch: 200
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7184299826622009

Current batch: 300
Current benign train accuracy: 0.6796875
Current benign train loss: 0.796712338924408

Total benign train accuarcy: 73.836
Total benign train loss: 292.18320164084435

[ Test epoch: 27 ]

Test accuarcy: 31.11
Test average loss: 0.05751319165229797
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.71875
Current benign train loss: 0.8605607748031616

Current batch: 100
Current benign train accuracy: 0.7578125
Current benign train lo


Current batch: 0
Current benign train accuracy: 0.7265625
Current benign train loss: 0.9095256328582764

Current batch: 100
Current benign train accuracy: 0.7109375
Current benign train loss: 0.99558424949646

Current batch: 200
Current benign train accuracy: 0.828125
Current benign train loss: 0.5037196278572083

Current batch: 300
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6619177460670471

Total benign train accuarcy: 77.162
Total benign train loss: 259.8895517885685

[ Test epoch: 40 ]

Test accuarcy: 35.89
Test average loss: 0.03150973465442657
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.7578125
Current benign train loss: 0.6501825451850891

Current batch: 100
Current benign train accuracy: 0.8046875
Current benign train loss: 0.6777743697166443

Current batch: 200
Current benign train accuracy: 0.734375
Current benign train loss: 0.6717773675918579

Current batch: 300
Current benign train accuracy: 0.7734375
Cur


Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.34012502431869507

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.42102620005607605

Total benign train accuarcy: 86.774
Total benign train loss: 151.51208671927452

[ Test epoch: 53 ]

Test accuarcy: 81.08
Test average loss: 0.0059035588502883915
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.32499581575393677

Current batch: 100
Current benign train accuracy: 0.84375
Current benign train loss: 0.40155553817749023

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.40167945623397827

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.5355638265609741

Total benign train accuarcy: 86.81
Total benign train loss: 149.69292291998863

[ Test epoch: 54 ]

Test accuarcy: 80.06
Test average loss: 0.006548506164550781


Total benign train accuarcy: 88.062
Total benign train loss: 136.0303412079811

[ Test epoch: 66 ]

Test accuarcy: 80.78
Test average loss: 0.0060360142052173615
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.1909351348876953

Current batch: 100
Current benign train accuracy: 0.8359375
Current benign train loss: 0.4425198435783386

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.3161725699901581

Current batch: 300
Current benign train accuracy: 0.828125
Current benign train loss: 0.5233517289161682

Total benign train accuarcy: 87.962
Total benign train loss: 135.58850707113743

[ Test epoch: 67 ]

Test accuarcy: 68.55
Test average loss: 0.010939229947328568
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.859375
Current benign train loss: 0.45738568902015686

Current batch: 100
Current benign train accuracy: 0.90625
Current benign tra


Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.23722872138023376

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.28414055705070496

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2801060378551483

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.23062805831432343

Total benign train accuarcy: 90.356
Total benign train loss: 111.80598758161068

[ Test epoch: 80 ]

Test accuarcy: 87.14
Test average loss: 0.0037936658218503
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.84375
Current benign train loss: 0.4092525541782379

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.33526039123535156

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.25108206272125244

Current batch: 300
Current benign train accuracy: 0.890625
Cu


Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.3530898094177246

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2442590296268463

Total benign train accuarcy: 90.586
Total benign train loss: 108.55093687772751

[ Test epoch: 93 ]

Test accuarcy: 87.32
Test average loss: 0.003767241068184376
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.21218164265155792

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2826854884624481

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.290134996175766

Current batch: 300
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3085322678089142

Total benign train accuarcy: 90.676
Total benign train loss: 106.64128226041794

[ Test epoch: 94 ]

Test accuarcy: 87.59
Test average loss: 0.0038395300403237342
Mod